In [4]:
import os
import tensorflow as tf
from zipfile import ZipFile
import tensorflow_hub as hub
from google.colab import drive
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB1
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
os.chdir("/content/drive/MyDrive/AcneDetection/")

In [7]:
# # Extract the zip file with the dataset.

# # Specify the path to your zip file
zip_file_path = 'data.zip'

# Specify the directory where you want to extract the contents
extract_to_directory = 'data/'

# Open the zip file
with ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extract_to_directory)
print("Zip file extracted successfully.")

Zip file extracted successfully.


In [8]:
weight_file = '/content/drive/MyDrive/AcneDetection/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Adjust inputs
pre_trained_model = InceptionV3(input_shape = (150, 150, 3),  include_top = False, weights = None)

In [9]:

pre_trained_model.load_weights(weight_file)


for layer in pre_trained_model.layers:
  layer.trainable = False

In [10]:
last_layer = pre_trained_model.get_layer('mixed5')
print('last layer output shape: ', last_layer.output_shape)

last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [11]:
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.MaxPooling2D()(last_output)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
# Add a dropout rate of 0.3
x = layers.Dropout(0.3)(x)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Append the dense network to the base model
acneModel = Model(pre_trained_model.input, x)

# Print the model summary
acneModel.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalization[0][0

In [13]:
acneModel.compile(optimizer = Adam(learning_rate=0.0001), loss = 'binary_crossentropy',  metrics = ['accuracy'])